### Data Pre-processing (detailed explanations are included in data-processing.ipynb)

In [1]:
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
import pandas as pd
from requests import get

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [4]:
body = soup.find('body')
#print(body.prettify())

In [5]:
body_content = body.find('div',class_='mw-parser-output').table.tbody
print(len(body_content))
data_list = []

580


In [6]:
for idx, tr in enumerate(body_content.find_all('tr')):
    if idx != 0:
        tds = tr.find_all('td')
        tds0 = tds[0]
        tds1 = tds[1].find('a')
        tds2 = tds[2].find('a')
        if tds1 == None and tds2 == None:   
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds[1].contents[0],
            'Neighborhood':tds[2].contents[0]
            })
        elif tds1 != None and tds2 == None:
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds1.text,
            'Neighborhood':tds[2].contents[0]
            })
        elif tds1 == None and tds2 != None:
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds[1].contents[0],
            'Neighborhood':tds2.text
            })
        else:
             data_list.append({
             'PostalCode': tds0.contents[0],
              'Borough': tds1.text,
              'Neighborhood':tds2.text
              })

In [7]:
df = pd.DataFrame(data_list,columns = ['PostalCode','Borough','Neighborhood'])

In [8]:
df = df[(True^df['Borough'].isin(['Not assigned']))]

In [9]:
for x in df['Neighborhood']:
    if x in ('Not assigned','Not assigned\n'):
        df.loc[df.Neighborhood == x,'Neighborhood']=df.loc[df.Neighborhood == x,'Borough']
    else:
        pass

In [10]:
df['Neighborhood']=df['Neighborhood'].str.replace('\n','')
#print(df.to_string()) #a way to display all rows of data

In [11]:
dataset = df.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()
dataset = dataset.join(df.set_index('PostalCode')['Borough'], on='PostalCode').drop_duplicates()  ##set PostalCode column as Index!
Torontodata = pd.DataFrame(dataset,columns = ['PostalCode','Borough','Neighborhood'])
Torontodata = Torontodata.sample(frac=1).reset_index(drop=True)

In [12]:
Torontodata.shape

(103, 3)

### Download geographical data from csv. (Geocoder package cannot be downloaded)

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [14]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data')

In [15]:
geodata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
Torontodata = Torontodata.join(geodata.set_index('PostalCode'), on='PostalCode')

In [16]:
CLIENT_ID = '5SVVVU4AU3C4K5F5UY53YABJQ1CJ3Y3U2DCDMVVWNS42DWE4' # your Foursquare ID
CLIENT_SECRET = 'QCA3IRJDRRV2KL14JLGAENMQRSURVCKPGJ2LCR3MKTBCMHYM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Work with only boroughs that contain the word Toronto. 

In [17]:
toronto_data = Torontodata[Torontodata['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4P,Central Toronto,Davisville North,43.712751,-79.390197
1,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
4,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


### Get the geographical coordinates of Toronto.

In [18]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now Let's start to explore a random neighborhood in our dataframe and get the top 100 venues that are within a radius of 500 meters.

In [20]:
toronto_data.loc[0, 'Neighborhood']

'Davisville North'

In [21]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Davisville North are 43.7127511, -79.3901975.


In [22]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [23]:
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sherwood Park,Park,43.716551,-79.387776
1,Summerhill Market North,Food & Drink Shop,43.715499,-79.392881
2,Homeway Restaurant & Brunch,Breakfast Spot,43.712641,-79.391557
3,Best Western Roehampton Hotel & Suites,Hotel,43.708878,-79.390880
4,Subway,Sandwich Place,43.708378,-79.390473


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


## Now let's explore all the neighborhoods in Toronto!

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Davisville North
The Annex,North Midtown,Yorkville
Lawrence Park
North Toronto West
Harbourfront East,Toronto Islands,Union Station
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
The Beaches
The Beaches West,India Bazaar
Cabbagetown,St. James Town
The Danforth West,Riverdale
Davisville
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Little Portugal,Trinity
Design Exchange,Toronto Dominion Centre
Moore Park,Summerhill East
Forest Hill North,Forest Hill West
Studio District
St. James Town
Brockton,Exhibition Place,Parkdale Village
Central Bay Street
Berczy Park
Stn A PO Boxes 25 The Esplanade
Harbord,University of Toronto
Harbourfront,Regent Park
Adelaide,King,Richmond
Commerce Court,Victoria Hotel
High Park,The Junction South
Church and Wellesley
Dovercourt Village,Dufferin
Business Reply Mail Processing Centre 969 Eastern
Rosedale
Parkdale,Roncesvalles
First Canadian Pla

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(1698, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
1,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
2,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
3,Davisville North,43.712751,-79.390197,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel
4,Davisville North,43.712751,-79.390197,Subway,43.708378,-79.390473,Sandwich Place


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


### Now analyze each neighborhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(38, 235)

### Create the new dataframe and display the top 20 venues for each neighborhood.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Clothing Store,Gym,Hotel,Restaurant,Bar,Bakery,Asian Restaurant,Breakfast Spot,Burger Joint,Pizza Place,Japanese Restaurant,Sushi Restaurant,Bookstore,Gastropub,Cosmetics Shop
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Pub,Beer Bar,Italian Restaurant,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop,Farmers Market,Café,Belgian Restaurant,Bistro,Jazz Club,Fish Market,Shopping Mall,Liquor Store,Basketball Stadium,Clothing Store
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Pet Store,Convenience Store,Climbing Gym,Caribbean Restaurant,Burrito Place,Stadium,Bar,Furniture / Home Store,Performing Arts Venue,Gym,Grocery Store,Gastropub,Diner,Electronics Store,Eastern European Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Comic Shop,Moving Target,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park,Smoke Shop,Fast Food Restaurant,Spa,Farmers Market,Park,Garden Center,Garden,Convenience Store,Grocery Store
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Plane,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Harbor / Marina,Airport,Airport Food Court,Dance Studio,Ethiopian Restaurant,Flea Market,Cuban Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Cupcake Shop,Fast Food Restaurant


### We can now cluster neighborhoods into 5 clustuers(we are quite close to finish, trust me.)

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2],
      dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Restaurant,Breakfast Spot,Sandwich Place,Burger Joint,Hotel,Food & Drink Shop,Gym,Park,Falafel Restaurant,Event Space,Doner Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Farmers Market,Donut Shop,Yoga Studio,Dog Run,Filipino Restaurant
1,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,2,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Jewish Restaurant,BBQ Joint,Liquor Store,Park,History Museum,Vegetarian / Vegan Restaurant,American Restaurant,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Lake,Dim Sum Restaurant,Swim School,Bus Line,Yoga Studio,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Doner Restaurant,Fish Market,Discount Store
3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Ice Cream Shop,Fast Food Restaurant,Park,Mexican Restaurant,Spa,Sandwich Place,Salon / Barbershop,Diner,Chinese Restaurant,Dessert Shop,Gift Shop,Health & Beauty Service,Cosmetics Shop,Pet Store,Gym / Fitness Center,Deli / Bodega
4,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,2,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery,Baseball Stadium,Bar,History Museum,Park,Music Venue,Sporting Goods Shop,Fried Chicken Joint,Sports Bar,Deli / Bodega,Salad Place


### Visualize data! (About to see something interesting in the end...almost the end)

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## At the end, we examine the clusters and determine the discriminating venue categories that distinguish each cluster.  </br>(Time to say goodbye but I will be always on the road. So see you again on the beautiful road ahead.)

### Cluster 1 (This cluster represents 'Outdoor Park' style venue)

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
32,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dim Sum Restaurant,Diner,Fish & Chips Shop,Dessert Shop
36,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Baby Store,Restaurant,Diner,Nightclub,Italian Restaurant,Deli / Bodega,Department Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 2 (This cluster represents more likely 'Sport' style venue)

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
16,Central Toronto,1,Trail,Playground,Gym,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio,Event Space,Discount Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Dog Run,Dim Sum Restaurant,Diner,Fish Market


### Cluster 3 (This cluster represents 'Cafe' style venue)

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Central Toronto,2,Restaurant,Breakfast Spot,Sandwich Place,Burger Joint,Hotel,Food & Drink Shop,Gym,Park,Falafel Restaurant,Event Space,Doner Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Farmers Market,Donut Shop,Yoga Studio,Dog Run,Filipino Restaurant
1,Central Toronto,2,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Jewish Restaurant,BBQ Joint,Liquor Store,Park,History Museum,Vegetarian / Vegan Restaurant,American Restaurant,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,Central Toronto,2,Park,Lake,Dim Sum Restaurant,Swim School,Bus Line,Yoga Studio,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Doner Restaurant,Fish Market,Discount Store
3,Central Toronto,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Ice Cream Shop,Fast Food Restaurant,Park,Mexican Restaurant,Spa,Sandwich Place,Salon / Barbershop,Diner,Chinese Restaurant,Dessert Shop,Gift Shop,Health & Beauty Service,Cosmetics Shop,Pet Store,Gym / Fitness Center,Deli / Bodega
4,Downtown Toronto,2,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery,Baseball Stadium,Bar,History Museum,Park,Music Venue,Sporting Goods Shop,Fried Chicken Joint,Sports Bar,Deli / Bodega,Salad Place
5,Downtown Toronto,2,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Burger Joint,Grocery Store,Gaming Cafe,Dim Sum Restaurant,Dessert Shop,Caribbean Restaurant,Farmers Market,Japanese Restaurant,Bubble Tea Shop,Record Shop,Brewery
6,Central Toronto,2,Convenience Store,Coffee Shop,Pub,Pizza Place,American Restaurant,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station,Vietnamese Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
8,East Toronto,2,Boutique,Coffee Shop,Pub,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Discount Store,Concert Hall,Diner,Dim Sum Restaurant
9,East Toronto,2,Sandwich Place,Liquor Store,Light Rail Station,Sushi Restaurant,Italian Restaurant,Brewery,Pub,Ice Cream Shop,Movie Theater,Steakhouse,Fish & Chips Shop,Burrito Place,Burger Joint,Pizza Place,Gym,Pet Store,Fast Food Restaurant,Park,Food & Drink Shop,Eastern European Restaurant
10,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Bakery,Italian Restaurant,Pub,Pizza Place,Pet Store,Pharmacy,Sandwich Place,Butcher,Caribbean Restaurant,Chinese Restaurant,Playground,Park,Beer Store,Deli / Bodega,Diner,Market,Liquor Store


### Cluster 4 (This cluster represents 'Indoor Service' style venue)

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
7,Central Toronto,3,Home Service,Garden,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Flea Market,Discount Store,Diner,Dim Sum Restaurant


### Cluster 5 (This cluster represents 'Restaurant' style venue)

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
17,Central Toronto,4,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Fish Market,Discount Store,Diner


## Thank you!